# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

A newer version of GraphLab Create (v1.8.1) is available! Your current version is v1.7.1.

You can use pip to upgrade the graphlab-create package. For more information see https://dato.com/products/create/upgrade.


In [33]:
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [124]:
sales1 = graphlab.SFrame('C:\Users\Jas\Documents\UofWash_ML\kc_house_data.gl/')


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

In [37]:
sales.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1.0,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2.0,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1.0,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1.0,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1.0,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1.0,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2.0,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1.0,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1.0,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2.0,0


# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [58]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 0.348580     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 0.381582     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

Find what features had non-zero weight.

In [59]:

model_all.coefficients.sort('value',ascending = False)


name,index,value
(intercept),None,274873.05595
bathrooms,None,8468.53108691
grade,None,842.068034898
sqft_living_sqrt,None,350.060553386
sqft_living,None,24.4207209824
sqft_above,None,20.0247224171
floors_square,None,0.0
yr_renovated,None,0.0
yr_built,None,0.0
sqft_basement,None,0.0


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [97]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate



Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [62]:
model_train = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty
                                              #,solver='fista', verbose = True
                                              #,max_iterations=30 
                                                #, convergence_threshold=1e-10
                                               )

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 9761
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000003  | 0.246413     | 6287414.310279     | 322429.554621 |
PROGRESS: | 2         | 3        | 0.000003  | 0.268413     | 6004129.870008     | 303352.364502 |
PROGRESS: | 3         | 4       

In [63]:

model_train.coefficients.sort('value',ascending = False)

name,index,value
waterfront,None,601905.594545
view,None,93312.8573119
bathrooms,None,25409.5889341
floors,None,21204.335467
(intercept),None,18993.4272128
floors_square,None,12915.5243361
bedrooms,None,7936.96767903
condition,None,6609.03571245
grade,None,6206.93999188
sqft_living_sqrt,None,1124.65021281


In [114]:

#for l1_penalty in np.logspace(1, 10, num=3):
for l1_penalty in  [10**1, 10**1.5, 10**2, 10**2.5,10**3, 10**3.5, 10**4, 10**4.5,
                    10**5, 10**5.5, 10**6, 10**6.5, 10**7]: 
    train_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False, solver ='fista',
                                              max_iterations = 30,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    validate_predict = train_model.predict(validation)
    RSS_validation = np.dot(validation['price'] - validate_predict, 
                            validation['price'] - validate_predict)
    print 'l1_penalty = %e' % l1_penalty
    print 'number of nonzeros = %d' % (train_model.coefficients['value']).nnz()
    #print train_model.coefficients.sort('value',ascending = False)
    #print predict
    print 'RSS_validation of %e' % l1_penalty + ' is ' + str(RSS_validation)
    print '\n'
    #plt.figure()
    #plot_poly_predictions(testing,train_model)
    #plt.title('LASSO, lambda = %.2e, # nonzeros = %d' % (l1_penalty, (train_model.coefficients['value']).nnz()))
    

l1_penalty = 1.000000e+01
number of nonzeros = 18
RSS_validation of 1.000000e+01 is 5.33943220017e+14


l1_penalty = 3.162278e+01
number of nonzeros = 18
RSS_validation of 3.162278e+01 is 5.33943220145e+14


l1_penalty = 1.000000e+02
number of nonzeros = 18
RSS_validation of 1.000000e+02 is 5.33943220552e+14


l1_penalty = 3.162278e+02
number of nonzeros = 18
RSS_validation of 3.162278e+02 is 5.33943221839e+14


l1_penalty = 1.000000e+03
number of nonzeros = 18
RSS_validation of 1.000000e+03 is 5.33943225907e+14


l1_penalty = 3.162278e+03
number of nonzeros = 18
RSS_validation of 3.162278e+03 is 5.33943238776e+14


l1_penalty = 1.000000e+04
number of nonzeros = 18
RSS_validation of 1.000000e+04 is 5.33943279498e+14


l1_penalty = 3.162278e+04
number of nonzeros = 18
RSS_validation of 3.162278e+04 is 5.33943408542e+14


l1_penalty = 1.000000e+05
number of nonzeros = 18
RSS_validation of 1.000000e+05 is 5.33943819333e+14


l1_penalty = 3.162278e+05
number of nonzeros = 18
RSS_validation

In [109]:
train_model.get_current_options

<bound method LinearRegression.get_current_options of Class                         : LinearRegression

Schema
------
Number of coefficients        : 18
Number of examples            : 9761
Number of feature columns     : 17
Number of unpacked features   : 17

Hyperparameters
---------------
L1 penalty                    : 10000000000.0
L2 penalty                    : 0.0

Training Summary
----------------
Solver                        : fista
Solver iterations             : 20
Solver status                 : TERMINATED: Iteration limit reached.
Training time (sec)           : 0.6647

Settings
--------
Residual sum of squares       : 1.40586724226e+15
Training RMSE                 : 379511.5515

Highest Positive Coefficients
-----------------------------
(intercept)                   : 543227.1768

Lowest Negative Coefficients
----------------------------
No Negative Coefficients      : 
>

In [85]:
train_model.coefficients

name,index,value
(intercept),None,564482.136844
bedrooms,None,0.0
bedrooms_square,None,0.0
bathrooms,None,0.0
sqft_living,None,0.0
sqft_living_sqrt,None,0.0
sqft_lot,None,0.0
sqft_lot_sqrt,None,0.0
floors,None,0.0
floors_square,None,0.0


In [87]:
for l1_penalty in np.logspace(1, 10, num=13):
    train_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    test_predict = train_model.predict(testing)
    RSS_testing = np.dot(testing['price'] - test_predict, testing['price'] - test_predict)
    print 'l1_penalty = %e' % l1_penalty
    print 'number of nonzeros = %d' % (train_model.coefficients['value']).nnz()
    #print train_model.coefficients.sort('value',ascending = False)
    #print predict
    print 'RSS_testing of %e' % l1_penalty + ' is ' + str(RSS_testing)
    print '\n'

l1_penalty = 1.000000e+01
number of nonzeros = 18
RSS_testing of 1.000000e+01 is 1.56983602382e+14


l1_penalty = 5.623413e+01
number of nonzeros = 18
RSS_testing of 5.623413e+01 is 1.56983602316e+14


l1_penalty = 3.162278e+02
number of nonzeros = 18
RSS_testing of 3.162278e+02 is 1.56983601949e+14


l1_penalty = 1.778279e+03
number of nonzeros = 18
RSS_testing of 1.778279e+03 is 1.56983599882e+14


l1_penalty = 1.000000e+04
number of nonzeros = 18
RSS_testing of 1.000000e+04 is 1.5698358826e+14


l1_penalty = 5.623413e+04
number of nonzeros = 18
RSS_testing of 5.623413e+04 is 1.56983522938e+14


l1_penalty = 3.162278e+05
number of nonzeros = 18
RSS_testing of 3.162278e+05 is 1.56983156601e+14


l1_penalty = 1.778279e+06
number of nonzeros = 18
RSS_testing of 1.778279e+06 is 1.56981128069e+14


l1_penalty = 1.000000e+07
number of nonzeros = 18
RSS_testing of 1.000000e+07 is 1.56972779669e+14


l1_penalty = 5.623413e+07
number of nonzeros = 18
RSS_testing of 5.623413e+07 is 1.569531805

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [ ]:
#1. RSS_validate is the smallest when l1_penalty is smallest
#2. l1_penalty =  5.623413e+07, last lambda before l1 start to crop coefficients

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [ ]:
18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [ ]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [88]:
l1_penalty_values = np.logspace(8, 10, num=20)
print l1_penalty_values

[  1.00000000e+08   1.27427499e+08   1.62377674e+08   2.06913808e+08
   2.63665090e+08   3.35981829e+08   4.28133240e+08   5.45559478e+08
   6.95192796e+08   8.85866790e+08   1.12883789e+09   1.43844989e+09
   1.83298071e+09   2.33572147e+09   2.97635144e+09   3.79269019e+09
   4.83293024e+09   6.15848211e+09   7.84759970e+09   1.00000000e+10]


Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [ ]:
l1_penalty_min = 7
l1_penalty_max = 10

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [ ]:
l1_penalty_values = np.logspace(l1_penalty_min,l1_penalty_max,20)


* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [96]:
l1_penalty_min = 9
l1_penalty_max = 10
l1_penalty_values = np.logspace(l1_penalty_min,l1_penalty_max,20)

for l1_penalty in l1_penalty_values:
    train_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    validation_predict = train_model.predict(validation)
    RSS_validation_sum = sum((validation['price'] - validation_predict)**2)
    RSS_validation_dot = np.dot(validation['price'] - validation_predict, validation['price'] - validation_predict)
    print 'l1_penalty = %e' % l1_penalty
    print 'number of nonzeros = %d' % (train_model.coefficients['value']).nnz()
    #print train_model.coefficients.sort('value',ascending = False)
    #print predict
    print 'RSS_validation of %e' % l1_penalty + ' is ' + str(RSS_validation_sum) + 'dot' + str(RSS_validation_dot)
    print '\n'
    
    

l1_penalty = 1.000000e+09
number of nonzeros = 16
RSS_validation of 1.000000e+09 is 6.88268359729e+14dot6.88268359729e+14


l1_penalty = 1.128838e+09
number of nonzeros = 15
RSS_validation of 1.128838e+09 is 7.01046815867e+14dot7.01046815867e+14


l1_penalty = 1.274275e+09
number of nonzeros = 15
RSS_validation of 1.274275e+09 is 7.16535679135e+14dot7.16535679135e+14


l1_penalty = 1.438450e+09
number of nonzeros = 15
RSS_validation of 1.438450e+09 is 7.37850622829e+14dot7.37850622829e+14


l1_penalty = 1.623777e+09
number of nonzeros = 15
RSS_validation of 1.623777e+09 is 7.66003337744e+14dot7.66003337744e+14


l1_penalty = 1.832981e+09
number of nonzeros = 13
RSS_validation of 1.832981e+09 is 7.9616310964e+14dot7.9616310964e+14


l1_penalty = 2.069138e+09
number of nonzeros = 13
RSS_validation of 2.069138e+09 is 8.28026516088e+14dot8.28026516088e+14


l1_penalty = 2.335721e+09
number of nonzeros = 12
RSS_validation of 2.335721e+09 is 8.69018172894e+14dot8.69018172894e+14


l1_penalty

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?